In [ ]:
import numpy as np
import anndata
import pandas as pd
import scanpy as sc
import scipy
import seaborn as sns

In [ ]:
Path="/home/mgander/mouse_atlas/data"
ts=['E3.5', 'E4.5', 'E5.25', 'E5.5', 'E6.25', 'E6.5', 'E6.75', 'E7.0', 'E7.25', 'E7.5', 'E7.75', 'E8.0', 'E8.25', 'E8.5a', 'E8.5b', 'E9.5', 'E10.5', 'E11.5', 'E12.5', 'E13.5']
cells=[32, 80, 100, 120, 400, 660, 1720, 4500, 8200, 15000, 30000, 60000, 73000, 90000, 90000, 200000, 1100000, 2600000, 6000000, 13000000]
Cell_number_dict={}
for i in range(20):
    Cell_number_dict[ts[i]]=cells[i]

In [ ]:
def single_cell_to_cell_type_transitions(ts0, ts1, method):
    adata=sc.read(f"{Path}/Comb_anndatas/adata_{ts0}_{ts1}.h5ad")
    del adata.raw
    day0,day1=sorted(set(adata.obs['day']))

    a0=adata[adata.obs['day']==day0].copy()
    a1=adata[adata.obs['day']==day1].copy()

    T=pd.read_pickle(f'{Path}/{method}_maps/Ts/Transitions_{ts0}_{ts1}.pkl')

    # Compute cell type transitions

    # First normalize w.r.t. to right marginal (doesn't really change much since we chose a huge tau2/lam2)
    T=T/T.sum(0)

    # Loop through cell types and sum up the wight of each submatrix
    cellstates0=sorted(set(a0.obs['cell_state']))
    cellstates1=sorted(set(a1.obs['cell_state']))

    CT=np.zeros((len(cellstates0), len(cellstates1)))

    for j,c1 in enumerate(cellstates1):
        inds1=list(a1[a1.obs['cell_state']==c1].obs.index)
        T0=T[inds1].copy()


        for i,c0 in enumerate(cellstates0):
            inds0=list(a0[a0.obs['cell_state']==c0].obs.index)

            CT[i,j]=T0.loc[inds0].sum().sum()
    CT=pd.DataFrame(data=CT, index=cellstates0, columns=cellstates1)
    CT.to_pickle(f'{Path}/{method}_maps/WOT_like_CTs/{ts0}_{ts1}_cell_type_transitions.pkl')

In [ ]:
for i in range(13):
    print(i)
    ts0=ts[i]
    ts1=ts[i+1]
    single_cell_to_cell_type_transitions(ts0, ts1, method='moscot')
    single_cell_to_cell_type_transitions(ts0, ts1, method='WOT')